In [6]:
import numpy as np
import pandas as pd

from keras.utils import to_categorical

In [2]:
# Load a single file as a numpy array
def load_file(filepath):
    df = pd.read_csv(filepath, header = None, delim_whitespace = True)
    return df.values

In [3]:
# Load a list of files into a 3D array of [samples, timesteps, features]
# One sample is one window of the time series data
# Each window has 128 time steps
# Each time step has 9 features

def load_group(filenames, prefix=''):
    loaded = list()
    for name in filenames:
        data = load_file(prefix + name)
        loaded.append(data)
    
    # Stack group so features are in the 3rd dimension
    loaded = np.dstack(loaded)
    return loaded

In [4]:
# Load a dataset group (train or test)
def load_dataset_group(group, prefix=''):
    filepath = prefix + group + '/Inertial Signals/'
    filenames = list()
    filenames += ['total_acc_x_'+group+'.txt', 'total_acc_y_'+group+'.txt', 'total_acc_z_'+group+'.txt']
    filenames += ['body_acc_x_'+group+'.txt', 'body_acc_y_'+group+'.txt', 'body_acc_z_'+group+'.txt']
    filenames += ['body_gyro_x_'+group+'.txt', 'body_gyro_y_'+group+'.txt', 'body_gyro_z_'+group+'.txt']

    x = load_group(filenames, filepath)
    y = load_file(prefix + group + f'/y_{group}.txt')
    return x, y

In [7]:
# Split dataset into train and test x and y elements
def load_dataset(prefix=''):
    train_x, train_y = load_dataset_group('train', prefix + 'HARDataset/')
    print(train_x.shape, train_y.shape)
    test_x, test_y = load_dataset_group('test', prefix + 'HARDataset/')
    print(test_x.shape, test_y.shape)

    # Zero-offset class values
    train_y = train_y - 1
    test_y = test_y - 1

    # One-hot encode y
    train_y = to_categorical(train_y)
    test_y = to_categorical(test_y)
    print(train_x.shape, train_y.shape, test_x.shape, test_y.shape)
    return train_x, train_y, test_x, test_y

In [8]:
train_x, train_y, test_x, test_y = load_dataset()
n_timesteps, n_features, n_outputs =train_x.shape[1], train_x.shape[2], train_y.shape[1]

(7352, 128, 9) (7352, 1)
(2947, 128, 9) (2947, 1)
(7352, 128, 9) (7352, 6) (2947, 128, 9) (2947, 6)


In [ ]:
# Train and evaluate a model
def evaluate_model(train_x, train_y, test_x, test_y):
    verbose = 0
    epochs = 10
    batch_size = 32
    n_timesteps, n_features, n_outputs = train_x.shape[1], train_x.shape[2], train_y.shape[1]

    model = Sequential()
    model.add(Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=(n_timesteps, n_features)))
    model.add(Conv1D(filters=64, kernel_size=3, activation='relu'))
    model.add(Dropout(0.5))
    model.add(MaxPooling1D(pool_size=2))
    model.add(Flatten())
    model.add(Dense(100, activation='relu'))
    model.add(Dense(n_outputs, activation='softmax'))
    model.compile(loss='categorial_crossentropy', optimizer='adam', metrics=['accuracy'])

    model.fit(train_x, train_y, epochs=epochs, batch_size=batch_size, verbose=verbose)

    _, accuracy = model.evaluate(test_x, test_y, batch_size=batch_size, verbose=0)
    return accuracy